# Task A : MongoDB Data Model

We have chosen an embedded model to follow for this assignment.

From the MongoDB documentation: 
"Embedded data models allow applications to store related pieces of information in the same database record. As a result, applications may need to issue fewer queries and updates to complete common operations."

We also have a one-to-many relationship in our data as each climate record might be associated with multiple hotspot records.Embedding (according to documentation) also provides better performances for read and update operations. 

Since this model affords us an increase in querying efficency, we believe that the embedded model is best suited for our data.

An example is as follows

```
climate_hist_doc = {"station": str("station": 948700,
                                    "date": 31/12/2016,
                                    "air_temperature_celcius": 19,
                                    "relative_humidity": 56.8,
                                    "windspeed_knots": 7.9,
                                    "max_windspeed" : 11.1,
                                    "precipitation" : 0.00I,
                                    "hotspots" : {"latitude" = -37.966,
                                                  "longitude" = 145.051,
                                                   "created_time" = 04:16:51,
                                                   "confidence" = 78,
                                                   "surface_temperature_celcius" = 68 }
                            }
                            
                         
```
The hotspot and climate data have been joined on the basis of date.
In hotspot data, we have changed datetime to created time as the date is already available in the climate records, and we believe that it is redundant to store date again. The time element has been extracted from the datetime and stored.

# Task B : Querying MongoDB using Pymongo

## Storing the data in MongoDB

In [22]:
#importing modules

import os
import csv
import pandas as pd
import datetime as dt

In [23]:
import pymongo
from pprint import pprint
from pymongo import MongoClient

# Method: connect on the default host and port
client = MongoClient ()

In [24]:
db = client.fit5148_db #initialise the database

In [25]:
db.climate_hist.drop() #drop collection 
climate_hist = db.climate_hist #create collection climate_hist

In [26]:
#we read in the files for storing
climate = pd.read_csv('climate_historic.csv')
hotspot = pd.read_csv('hotspot_historic.csv')

In [27]:
for i in range(len(climate)): #iterating through every row in the climate_historic.csv
    
    date = climate['date'][i]  #storing the date of the climate record
    latitude = [] #empty list for hotspot latitudes
    longitude = [] #empty list for hotspot longitudes
    datetime = [] #empty list for hotspot datetimes
    confidence = [] #empty list for hotspot confidence
    surface_temperature_celcius = []  #empty list for hotspot surface temperatures
    hots = [] #creating an empty list for hotspot data
    for j in range(len(hotspot)): #iterating through every row of hotspot_historic.csv
        if date == hotspot['date'][j]: #comparing the climate record date with hotspot record date, and if true, then hotspot records are recorded
            latitude.append(hotspot['latitude'][j]) 
            longitude.append(hotspot['longitude'][j]) 
            f = dt.datetime.strptime(hotspot['datetime'][j], '%Y-%m-%dT%H:%M:%S')
            g = f.strftime("%X") #extracting only the time value from hostpot datetime
            datetime.append(g)
            confidence.append(hotspot['confidence'][j])
            surface_temperature_celcius.append(hotspot['surface_temperature_celcius'][j])
        
    for k in range(len(latitude)): #a loop to iterate through every hotspot data, "latitude" is arbitrary
        
        dic = {} #empty dict to continually store
        dic['latitude'] = float(latitude[k])
        dic['longitude'] = float(longitude[k])
        dic['created_time'] = str(datetime[k])
        dic['confidence'] = int(confidence[k])
        dic['surface_temperature_celcius']= int(surface_temperature_celcius[k])
    
        hots.append(dic) #inserting hotspot data into empty list
    
    if not hots: #if the list for hotspot data is empty
        
        climate_hist_doc = {"station": str(climate['station'][i]),
                                    "date": str(climate['date'][i]),
                                    "air_temperature_celcius": int(climate['air_temperature_celcius'][i]),
                                    "relative_humidity": float(climate['relative_humidity'][i]),
                                    "windspeed_knots": float(climate['windspeed_knots'][i]),
                                    "max_windspeed" : float(climate['max_wind_speed'][i]),
                                    "precipitation" : str(climate['precipitation '][i])
                            }
        
        climate_hist.insert_one(climate_hist_doc) #inserting document into climate
        
    else: #if hotspot data is available
        
        climate_hist_doc = {"station": str(climate['station'][i]),
                                    "date": str(climate['date'][i]),
                                    "air_temperature_celcius": int(climate['air_temperature_celcius'][i]),
                                    "relative_humidity": float(climate['relative_humidity'][i]),
                                    "windspeed_knots": float(climate['windspeed_knots'][i]),
                                    "max_windspeed" : float(climate['max_wind_speed'][i]),
                                    "precipitation" : str(climate['precipitation '][i]),                               
                                    "hotspots" : hots #inserting hotspot data
                            }
        
        climate_hist.insert_one(climate_hist_doc)
    
    pprint(climate_hist_doc) #to see if records have been stored correctly


{'_id': ObjectId('5ce73deb9343690d63baee9d'),
 'air_temperature_celcius': 19,
 'date': '31/12/2016',
 'max_windspeed': 11.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.8,
 'station': '948700',
 'windspeed_knots': 7.9}
{'_id': ObjectId('5ce73deb9343690d63baee9e'),
 'air_temperature_celcius': 15,
 'date': '2/01/2017',
 'max_windspeed': 13.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 50.7,
 'station': '948700',
 'windspeed_knots': 9.2}
{'_id': ObjectId('5ce73deb9343690d63baee9f'),
 'air_temperature_celcius': 16,
 'date': '3/01/2017',
 'max_windspeed': 15.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 53.6,
 'station': '948700',
 'windspeed_knots': 8.1}
{'_id': ObjectId('5ce73deb9343690d63baeea0'),
 'air_temperature_celcius': 24,
 'date': '4/01/2017',
 'max_windspeed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.6,
 'station': '948700',
 'windspeed_knots': 7.7}
{'_id': ObjectId('5ce73deb9343690d63baeea1'),
 'air_temperature_celcius': 24,
 'date': '5/0

{'_id': ObjectId('5ce73ded9343690d63baeec2'),
 'air_temperature_celcius': 20,
 'date': '7/02/2017',
 'max_windspeed': 22.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 57.4,
 'station': '948700',
 'windspeed_knots': 10.9}
{'_id': ObjectId('5ce73ded9343690d63baeec3'),
 'air_temperature_celcius': 31,
 'date': '8/02/2017',
 'max_windspeed': 23.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.9,
 'station': '948700',
 'windspeed_knots': 14.1}
{'_id': ObjectId('5ce73ded9343690d63baeec4'),
 'air_temperature_celcius': 22,
 'date': '9/02/2017',
 'max_windspeed': 18.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 62.7,
 'station': '948700',
 'windspeed_knots': 7.6}
{'_id': ObjectId('5ce73ded9343690d63baeec5'),
 'air_temperature_celcius': 21,
 'date': '10/02/2017',
 'max_windspeed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 60.4,
 'station': '948700',
 'windspeed_knots': 6.8}
{'_id': ObjectId('5ce73ded9343690d63baeec6'),
 'air_temperature_celcius': 19,
 'date': '1

{'_id': ObjectId('5ce73def9343690d63baeee3'),
 'air_temperature_celcius': 21,
 'date': '12/03/2017',
 'hotspots': [{'confidence': 94,
               'created_time': '04:33:50',
               'latitude': -37.4157,
               'longitude': 147.0084,
               'surface_temperature_celcius': 105},
              {'confidence': 84,
               'created_time': '04:29:50',
               'latitude': -37.0193,
               'longitude': 148.1459,
               'surface_temperature_celcius': 71},
              {'confidence': 100,
               'created_time': '04:28:20',
               'latitude': -37.4229,
               'longitude': 147.02700000000002,
               'surface_temperature_celcius': 99},
              {'confidence': 80,
               'created_time': '04:28:00',
               'latitude': -37.0055,
               'longitude': 148.1582,
               'surface_temperature_celcius': 68},
              {'confidence': 85,
               'created_time': '04:27:20',
   

{'_id': ObjectId('5ce73df09343690d63baeeeb'),
 'air_temperature_celcius': 23,
 'date': '20/03/2017',
 'max_windspeed': 15.9,
 'precipitation': ' 0.08G',
 'relative_humidity': 63.8,
 'station': '948701',
 'windspeed_knots': 7.1}
{'_id': ObjectId('5ce73df09343690d63baeeec'),
 'air_temperature_celcius': 20,
 'date': '21/03/2017',
 'max_windspeed': 14.0,
 'precipitation': ' 0.63G',
 'relative_humidity': 67.4,
 'station': '948701',
 'windspeed_knots': 8.5}
{'_id': ObjectId('5ce73df09343690d63baeeed'),
 'air_temperature_celcius': 18,
 'date': '22/03/2017',
 'max_windspeed': 21.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 57.5,
 'station': '948701',
 'windspeed_knots': 16.1}
{'_id': ObjectId('5ce73df09343690d63baeeee'),
 'air_temperature_celcius': 18,
 'date': '23/03/2017',
 'max_windspeed': 19.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 55.6,
 'station': '948701',
 'windspeed_knots': 12.8}
{'_id': ObjectId('5ce73df09343690d63baeeef'),
 'air_temperature_celcius': 20,
 'date':

               'longitude': 141.9382,
               'surface_temperature_celcius': 52},
              {'confidence': 88,
               'created_time': '04:28:30',
               'latitude': -36.4573,
               'longitude': 142.2323,
               'surface_temperature_celcius': 63},
              {'confidence': 87,
               'created_time': '04:28:30',
               'latitude': -37.4996,
               'longitude': 142.5966,
               'surface_temperature_celcius': 62},
              {'confidence': 79,
               'created_time': '04:28:00',
               'latitude': -34.4501,
               'longitude': 141.4699,
               'surface_temperature_celcius': 52},
              {'confidence': 97,
               'created_time': '04:27:40',
               'latitude': -37.9452,
               'longitude': 143.5834,
               'surface_temperature_celcius': 80},
              {'confidence': 100,
               'created_time': '04:27:20',
               'latitude':

...

{'_id': ObjectId('5ce73e069343690d63baefc1'),
 'air_temperature_celcius': 13,
 'date': '20/10/2017',
 'hotspots': [{'confidence': 80,
               'created_time': '04:41:19',
               'latitude': -37.586,
               'longitude': 141.27100000000002,
               'surface_temperature_celcius': 46},
              {'confidence': 82,
               'created_time': '04:41:19',
               'latitude': -37.588,
               'longitude': 141.259,
               'surface_temperature_celcius': 50},
              {'confidence': 65,
               'created_time': '00:27:12',
               'latitude': -37.296,
               'longitude': 144.386,
               'surface_temperature_celcius': 54}],
 'max_windspeed': 16.9,
 'precipitation': ' 0.00G',
 'relative_humidity': 42.0,
 'station': '948702',
 'windspeed_knots': 11.4}
{'_id': ObjectId('5ce73e069343690d63baefc2'),
 'air_temperature_celcius': 12,
 'date': '21/10/2017',
 'hotspots': [{'confidence': 77,
               'created_t

{'_id': ObjectId('5ce73e079343690d63baefd5'),
 'air_temperature_celcius': 14,
 'date': '9/11/2017',
 'hotspots': [{'confidence': 68,
               'created_time': '04:16:56',
               'latitude': -36.759,
               'longitude': 145.179,
               'surface_temperature_celcius': 52},
              {'confidence': 62,
               'created_time': '04:16:56',
               'latitude': -36.758,
               'longitude': 145.19,
               'surface_temperature_celcius': 51},
              {'confidence': 75,
               'created_time': '04:16:53',
               'latitude': -37.213,
               'longitude': 142.941,
               'surface_temperature_celcius': 51},
              {'confidence': 85,
               'created_time': '04:16:48',
               'latitude': -37.479,
               'longitude': 143.358,
               'surface_temperature_celcius': 59},
              {'confidence': 72,
               'created_time': '04:16:48',
               'latitude'

{'_id': ObjectId('5ce73e089343690d63baefe2'),
 'air_temperature_celcius': 23,
 'date': '22/11/2017',
 'hotspots': [{'confidence': 84,
               'created_time': '03:46:00',
               'latitude': -37.978,
               'longitude': 145.623,
               'surface_temperature_celcius': 59},
              {'confidence': 88,
               'created_time': '03:46:00',
               'latitude': -37.976,
               'longitude': 145.649,
               'surface_temperature_celcius': 64}],
 'max_windspeed': 13.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 49.6,
 'station': '948702',
 'windspeed_knots': 6.3}
{'_id': ObjectId('5ce73e089343690d63baefe3'),
 'air_temperature_celcius': 24,
 'date': '23/11/2017',
 'hotspots': [{'confidence': 84,
               'created_time': '04:28:59',
               'latitude': -37.946,
               'longitude': 144.342,
               'surface_temperature_celcius': 60},
              {'confidence': 93,
               'created_time': '04:28

{'_id': ObjectId('5ce73e099343690d63baefee'),
 'air_temperature_celcius': 14,
 'date': '4/12/2017',
 'max_windspeed': 15.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 48.2,
 'station': '948702',
 'windspeed_knots': 9.1}
{'_id': ObjectId('5ce73e099343690d63baefef'),
 'air_temperature_celcius': 17,
 'date': '5/12/2017',
 'max_windspeed': 22.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 53.0,
 'station': '948702',
 'windspeed_knots': 14.6}
{'_id': ObjectId('5ce73e099343690d63baeff0'),
 'air_temperature_celcius': 16,
 'date': '6/12/2017',
 'max_windspeed': 15.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 51.7,
 'station': '948702',
 'windspeed_knots': 7.4}
{'_id': ObjectId('5ce73e099343690d63baeff1'),
 'air_temperature_celcius': 15,
 'date': '7/12/2017',
 'max_windspeed': 13.0,
 'precipitation': ' 1.26G',
 'relative_humidity': 51.0,
 'station': '948702',
 'windspeed_knots': 9.0}
{'_id': ObjectId('5ce73e099343690d63baeff2'),
 'air_temperature_celcius': 15,
 'date': '8/1

{'_id': ObjectId('5ce73e0a9343690d63baeffb'),
 'air_temperature_celcius': 21,
 'date': '17/12/2017',
 'max_windspeed': 13.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 58.7,
 'station': '948702',
 'windspeed_knots': 7.8}
{'_id': ObjectId('5ce73e0a9343690d63baeffc'),
 'air_temperature_celcius': 25,
 'date': '18/12/2017',
 'max_windspeed': 8.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 60.9,
 'station': '948702',
 'windspeed_knots': 6.3}
{'_id': ObjectId('5ce73e0a9343690d63baeffd'),
 'air_temperature_celcius': 21,
 'date': '19/12/2017',
 'max_windspeed': 22.9,
 'precipitation': ' 0.67G',
 'relative_humidity': 57.4,
 'station': '948702',
 'windspeed_knots': 10.7}
{'_id': ObjectId('5ce73e0a9343690d63baeffe'),
 'air_temperature_celcius': 17,
 'date': '20/12/2017',
 'max_windspeed': 18.1,
 'precipitation': ' 0.00G',
 'relative_humidity': 51.2,
 'station': '948702',
 'windspeed_knots': 9.9}
{'_id': ObjectId('5ce73e0a9343690d63baefff'),
 'air_temperature_celcius': 17,
 'date': '

In [28]:
db.list_collection_names() #checking if the collection has been stored correctly

['climate_hist', 'week12', 'climate_streaming']

## Querying MongoDb

a. Find the climate data on 10th December 2017

In [29]:
results = climate_hist.find({'date':'10/12/2017'})
for doc in results:
    pprint(doc)

{'_id': ObjectId('5ce73e0a9343690d63baeff4'),
 'air_temperature_celcius': 17,
 'date': '10/12/2017',
 'hotspots': [{'confidence': 50,
               'created_time': '00:57:59',
               'latitude': -37.986999999999995,
               'longitude': 144.005,
               'surface_temperature_celcius': 38},
              {'confidence': 67,
               'created_time': '00:56:57',
               'latitude': -34.289,
               'longitude': 141.71200000000002,
               'surface_temperature_celcius': 54}],
 'max_windspeed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 53.5,
 'station': '948702',
 'windspeed_knots': 7.3}


b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C 

In [30]:
result = climate_hist.aggregate(
       [ { "$unwind" : "$hotspots" },
         {"$match":{"$and":[ { "hotspots.surface_temperature_celcius": { "$gt": 65, "$lt": 100 } }]}},
         { "$sort" : { "hotspots.surface_temperature_celcius" : -1} }
       
       ]
          )
for docs in result:
    pprint(docs)

{'_id': ObjectId('5ce73def9343690d63baeee3'),
 'air_temperature_celcius': 21,
 'date': '12/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:28:20',
              'latitude': -37.4229,
              'longitude': 147.02700000000002,
              'surface_temperature_celcius': 99},
 'max_windspeed': 11.1,
 'precipitation': ' 0.00G',
 'relative_humidity': 58.1,
 'station': '948701',
 'windspeed_knots': 5.3}
{'_id': ObjectId('5ce73df09343690d63baeef0'),
 'air_temperature_celcius': 23,
 'date': '25/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '03:56:40',
              'latitude': -37.8881,
              'longitude': 145.8229,
              'surface_temperature_celcius': 99},
 'max_windspeed': 15.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.9,
 'station': '948701',
 'windspeed_knots': 8.0}
{'_id': ObjectId('5ce73df19343690d63baeefc'),
 'air_temperature_celcius': 19,
 'date': '6/04/2017',
 'hotspots': {'confidence': 100,
     

 'hotspots': {'confidence': 100,
              'created_time': '04:26:30',
              'latitude': -38.0795,
              'longitude': 143.2449,
              'surface_temperature_celcius': 91},
 'max_windspeed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 54.9,
 'station': '948701',
 'windspeed_knots': 5.9}
{'_id': ObjectId('5ce73dfa9343690d63baef17'),
 'air_temperature_celcius': 10,
 'date': '3/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:03:00',
              'latitude': -36.3459,
              'longitude': 141.5586,
              'surface_temperature_celcius': 91},
 'max_windspeed': 8.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 43.1,
 'station': '948701',
 'windspeed_knots': 4.7}
{'_id': ObjectId('5ce73df09343690d63baeef3'),
 'air_temperature_celcius': 18,
 'date': '28/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:26:50',
              'latitude': -37.5012,
              'longitude': 142.5853,
   

 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'_id': ObjectId('5ce73dfa9343690d63baef18'),
 'air_temperature_celcius': 10,
 'date': '4/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:45:10',
              'latitude': -36.664,
              'longitude': 142.8802,
              'surface_temperature_celcius': 87},
 'max_windspeed': 7.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 43.3,
 'station': '948701',
 'windspeed_knots': 2.6}
{'_id': ObjectId('5ce73dfa9343690d63baef18'),
 'air_temperature_celcius': 10,
 'date': '4/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '00:25:10',
              'latitude': -37.8412,
              'longitude': 147.2536,
              'surface_temperature_celcius': 87},
 'max_windspeed': 7.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 43.3,
 'station': '948701',
 'windspeed_knots': 2.6}
{'_id': ObjectId('5ce73dfc9343690d63baef23'),
 'air_temperature_celcius': 10,


              'longitude': 142.4642,
              'surface_temperature_celcius': 81},
 'max_windspeed': 13.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 50.6,
 'station': '948701',
 'windspeed_knots': 6.0}
{'_id': ObjectId('5ce73df19343690d63baeefa'),
 'air_temperature_celcius': 16,
 'date': '4/04/2017',
 'hotspots': {'confidence': 92,
              'created_time': '04:33:50',
              'latitude': -36.5989,
              'longitude': 141.9771,
              'surface_temperature_celcius': 81},
 'max_windspeed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': '948701',
 'windspeed_knots': 5.4}
{'_id': ObjectId('5ce73df19343690d63baeefc'),
 'air_temperature_celcius': 19,
 'date': '6/04/2017',
 'hotspots': {'confidence': 97,
              'created_time': '04:20:40',
              'latitude': -37.7617,
              'longitude': 143.08100000000002,
              'surface_temperature_celcius': 81},
 'max_windspeed': 15.9,
 'precipitation': ' 0.00I',
 'r

 'date': '11/05/2017',
 'hotspots': {'confidence': 84,
              'created_time': '04:50:40',
              'latitude': -37.3656,
              'longitude': 141.3903,
              'surface_temperature_celcius': 78},
 'max_windspeed': 6.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 45.3,
 'station': '948701',
 'windspeed_knots': 2.5}
{'_id': ObjectId('5ce73dfc9343690d63baef23'),
 'air_temperature_celcius': 10,
 'date': '15/05/2017',
 'hotspots': {'confidence': 96,
              'created_time': '04:34:30',
              'latitude': -37.253,
              'longitude': 144.3681,
              'surface_temperature_celcius': 78},
 'max_windspeed': 8.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.0,
 'station': '948701',
 'windspeed_knots': 5.2}
{'_id': ObjectId('5ce73dfc9343690d63baef23'),
 'air_temperature_celcius': 10,
 'date': '15/05/2017',
 'hotspots': {'confidence': 90,
              'created_time': '04:26:20',
              'latitude': -36.8616,
              'longi

 'windspeed_knots': 5.9}
{'_id': ObjectId('5ce73df79343690d63baef03'),
 'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 95,
              'created_time': '04:26:30',
              'latitude': -37.8906,
              'longitude': 146.2035,
              'surface_temperature_celcius': 75},
 'max_windspeed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 54.9,
 'station': '948701',
 'windspeed_knots': 5.9}
{'_id': ObjectId('5ce73df89343690d63baef08'),
 'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspots': {'confidence': 94,
              'created_time': '04:44:50',
              'latitude': -36.092,
              'longitude': 141.7306,
              'surface_temperature_celcius': 75},
 'max_windspeed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'_id': ObjectId('5ce73dfb9343690d63baef1f'),
 'air_temperature_celcius': 9,
 'date': '11/05/2017',
 'hotspots': {'confidence': 

              'surface_temperature_celcius': 72},
 'max_windspeed': 15.9,
 'precipitation': ' 0.31G',
 'relative_humidity': 53.5,
 'station': '948701',
 'windspeed_knots': 7.2}
{'_id': ObjectId('5ce73df99343690d63baef0d'),
 'air_temperature_celcius': 17,
 'date': '23/04/2017',
 'hotspots': {'confidence': 93,
              'created_time': '05:03:10',
              'latitude': -36.9111,
              'longitude': 142.692,
              'surface_temperature_celcius': 72},
 'max_windspeed': 7.0,
 'precipitation': ' 0.01G',
 'relative_humidity': 59.3,
 'station': '948701',
 'windspeed_knots': 4.2}
{'_id': ObjectId('5ce73dfa9343690d63baef15'),
 'air_temperature_celcius': 14,
 'date': '1/05/2017',
 'hotspots': {'confidence': 70,
              'created_time': '04:14:20',
              'latitude': -36.9303,
              'longitude': 143.1034,
              'surface_temperature_celcius': 72},
 'max_windspeed': 20.0,
 'precipitation': ' 0.12G',
 'relative_humidity': 49.2,
 'station': '948701',
 

 'precipitation': ' 0.00I',
 'relative_humidity': 58.3,
 'station': '948701',
 'windspeed_knots': 7.1}
{'_id': ObjectId('5ce73df09343690d63baeef1'),
 'air_temperature_celcius': 23,
 'date': '26/03/2017',
 'hotspots': {'confidence': 91,
              'created_time': '04:39:10',
              'latitude': -38.0674,
              'longitude': 142.965,
              'surface_temperature_celcius': 68},
 'max_windspeed': 26.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 60.6,
 'station': '948701',
 'windspeed_knots': 10.1}
{'_id': ObjectId('5ce73df19343690d63baeefa'),
 'air_temperature_celcius': 16,
 'date': '4/04/2017',
 'hotspots': {'confidence': 64,
              'created_time': '04:39:50',
              'latitude': -36.6281,
              'longitude': 142.4349,
              'surface_temperature_celcius': 68},
 'max_windspeed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': '948701',
 'windspeed_knots': 5.4}
{'_id': ObjectId('5ce73df19343690d63baeefc'),
 '

              'surface_temperature_celcius': 67},
 'max_windspeed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'_id': ObjectId('5ce73df89343690d63baef08'),
 'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspots': {'confidence': 90,
              'created_time': '04:44:50',
              'latitude': -37.7756,
              'longitude': 143.1388,
              'surface_temperature_celcius': 67},
 'max_windspeed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'_id': ObjectId('5ce73df99343690d63baef0a'),
 'air_temperature_celcius': 20,
 'date': '20/04/2017',
 'hotspots': {'confidence': 90,
              'created_time': '04:32:50',
              'latitude': -36.1303,
              'longitude': 146.3606,
              'surface_temperature_celcius': 67},
 'max_windspeed': 15.9,
 'precipitation': ' 0.31G',
 'relative_humidity': 53.5,
 'station': '948701',


              'longitude': 148.101,
              'surface_temperature_celcius': 66},
 'max_windspeed': 23.9,
 'precipitation': ' 0.16G',
 'relative_humidity': 44.0,
 'station': '948702',
 'windspeed_knots': 14.4}
{'_id': ObjectId('5ce73e0a9343690d63baeffa'),
 'air_temperature_celcius': 18,
 'date': '16/12/2017',
 'hotspots': {'confidence': 90,
              'created_time': '04:34:57',
              'latitude': -37.624,
              'longitude': 149.314,
              'surface_temperature_celcius': 66},
 'max_windspeed': 13.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 53.7,
 'station': '948702',
 'windspeed_knots': 9.0}


c. Find date, surface temperature (°C), air temperature (°C), relative humidity and  max wind speed on 15th and 16th of December 2017.

In [31]:
climate = db.climate_hist
results = climate.aggregate([
   {"$match" : { "$or": [ {"date":"15/12/2017"}, {"date":"16/12/2017"} ]}},       
   {"$project" : {"_id":0,"date":1,"hotspots.surface_temperature_celcius":1,"air_temperature_celcius":1,"relative_humidity":1,"max_windspeed":1}}

])

for doc in results:
  #  q2c.insert_one(doc)
    pprint(doc)


{'air_temperature_celcius': 18,
 'date': '15/12/2017',
 'hotspots': [{'surface_temperature_celcius': 42},
              {'surface_temperature_celcius': 36},
              {'surface_temperature_celcius': 38},
              {'surface_temperature_celcius': 40}],
 'max_windspeed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': '16/12/2017',
 'hotspots': [{'surface_temperature_celcius': 43},
              {'surface_temperature_celcius': 33},
              {'surface_temperature_celcius': 54},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 75},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 66},
              {'surface_temperature_celcius': 56},
              {'surface_temperature_celcius': 60},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 48},
              {'surfa

d. Find datetime, air temperature(°C), surface temperature(°C) and confidence when the confidence is between 80 and 100.  

In [32]:

result = climate_hist.aggregate(
       [ { "$unwind" : "$hotspots" },
         {"$match":{"$and":[ { "hotspots.confidence": { "$gte": 80, "$lte": 100 } }]}},
         { "$sort" : { "hotspots.confidence" : -1} },
        {"$project" : {"_id":0,"date":1,"hotspots.created_time":1,"hotspots.surface_temperature_celcius":1,"air_temperature_celcius":1, "hotspots.confidence":1}}
       
       ]
          )
for docs in result:
    pprint(docs)




{'air_temperature_celcius': 19,
 'date': '10/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:48:40',
              'surface_temperature_celcius': 105}}
{'air_temperature_celcius': 19,
 'date': '10/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:46:20',
              'surface_temperature_celcius': 109}}
{'air_temperature_celcius': 21,
 'date': '12/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:28:20',
              'surface_temperature_celcius': 99}}
{'air_temperature_celcius': 21,
 'date': '18/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '03:50:50',
              'surface_temperature_celcius': 121}}
{'air_temperature_celcius': 24,
 'date': '19/03/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:32:50',
              'surface_temperature_celcius': 91}}
{'air_temperature_celcius': 24,
 'date': '19/03/2017',
 'hotspots': {'confidence': 100,
         

 'date': '13/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:27:30',
              'surface_temperature_celcius': 88}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:27:20',
              'surface_temperature_celcius': 113}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:27:20',
              'surface_temperature_celcius': 95}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:27:20',
              'surface_temperature_celcius': 92}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:27:00',
              'surface_temperature_celcius': 101}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:26:50',


{'air_temperature_celcius': 9,
 'date': '13/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:45:40',
              'surface_temperature_celcius': 102}}
{'air_temperature_celcius': 9,
 'date': '13/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:41:50',
              'surface_temperature_celcius': 98}}
{'air_temperature_celcius': 9,
 'date': '13/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:40:20',
              'surface_temperature_celcius': 120}}
{'air_temperature_celcius': 9,
 'date': '13/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:40:10',
              'surface_temperature_celcius': 90}}
{'air_temperature_celcius': 9,
 'date': '13/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:38:50',
              'surface_temperature_celcius': 93}}
{'air_temperature_celcius': 9,
 'date': '13/05/2017',
 'hotspots': {'confidence': 100,
              'c

{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 98,
              'created_time': '04:31:00',
              'surface_temperature_celcius': 82}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 98,
              'created_time': '04:27:40',
              'surface_temperature_celcius': 82}}
{'air_temperature_celcius': 12,
 'date': '15/04/2017',
 'hotspots': {'confidence': 98,
              'created_time': '04:20:40',
              'surface_temperature_celcius': 82}}
{'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspots': {'confidence': 98,
              'created_time': '04:48:10',
              'surface_temperature_celcius': 82}}
{'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspots': {'confidence': 98,
              'created_time': '04:45:00',
              'surface_temperature_celcius': 82}}
{'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspots': {'confidence': 98,
              'cre

 'hotspots': {'confidence': 95,
              'created_time': '04:28:10',
              'surface_temperature_celcius': 77}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 95,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 76}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 95,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 75}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 95,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 77}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 95,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 75}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 95,
              'created_time': '00:07:00',
              'surface_temperat

...

              'created_time': '12:50:40',
              'surface_temperature_celcius': 42}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 89,
              'created_time': '04:36:20',
              'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 89,
              'created_time': '04:35:40',
              'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 89,
              'created_time': '04:33:20',
              'surface_temperature_celcius': 66}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 89,
              'created_time': '04:30:50',
              'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 89,
              'created_time': '04:30:50',
              'surface_temperature_celcius': 64}}
{'air_tempera

 'date': '29/04/2017',
 'hotspots': {'confidence': 87,
              'created_time': '04:33:00',
              'surface_temperature_celcius': 90}}
{'air_temperature_celcius': 14,
 'date': '1/05/2017',
 'hotspots': {'confidence': 87,
              'created_time': '04:14:20',
              'surface_temperature_celcius': 117}}
{'air_temperature_celcius': 10,
 'date': '3/05/2017',
 'hotspots': {'confidence': 87,
              'created_time': '04:02:20',
              'surface_temperature_celcius': 63}}
{'air_temperature_celcius': 10,
 'date': '4/05/2017',
 'hotspots': {'confidence': 87,
              'created_time': '04:48:10',
              'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 10,
 'date': '4/05/2017',
 'hotspots': {'confidence': 87,
              'created_time': '04:46:50',
              'surface_temperature_celcius': 63}}
{'air_temperature_celcius': 10,
 'date': '4/05/2017',
 'hotspots': {'confidence': 87,
              'created_time': '04:45:30',
            

 'hotspots': {'confidence': 86,
              'created_time': '04:47:31',
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 18,
 'date': '11/11/2017',
 'hotspots': {'confidence': 86,
              'created_time': '04:04:25',
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 17,
 'date': '9/12/2017',
 'hotspots': {'confidence': 86,
              'created_time': '00:14:33',
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 28,
 'date': '27/12/2017',
 'hotspots': {'confidence': 86,
              'created_time': '00:02:14',
              'surface_temperature_celcius': 67}}
{'air_temperature_celcius': 20,
 'date': '6/03/2017',
 'hotspots': {'confidence': 85,
              'created_time': '05:06:20',
              'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 21,
 'date': '12/03/2017',
 'hotspots': {'confidence': 85,
              'created_time': '04:27:20',
              'surface_temperatur

 'hotspots': {'confidence': 83,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 83,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 83,
              'created_time': '04:26:30',
              'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 15,
 'date': '16/04/2017',
 'hotspots': {'confidence': 83,
              'created_time': '05:07:40',
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 15,
 'date': '16/04/2017',
 'hotspots': {'confidence': 83,
              'created_time': '04:56:50',
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 13,
 'date': '17/04/2017',
 'hotspots': {'confidence': 83,
              'created_time': '04:03:50',
              'surface_temperat

{'air_temperature_celcius': 19,
 'date': '6/04/2017',
 'hotspots': {'confidence': 81,
              'created_time': '04:20:40',
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 19,
 'date': '6/04/2017',
 'hotspots': {'confidence': 81,
              'created_time': '04:20:40',
              'surface_temperature_celcius': 61}}
{'air_temperature_celcius': 19,
 'date': '6/04/2017',
 'hotspots': {'confidence': 81,
              'created_time': '04:20:40',
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 21,
 'date': '7/04/2017',
 'hotspots': {'confidence': 81,
              'created_time': '05:03:10',
              'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 14,
 'date': '11/04/2017',
 'hotspots': {'confidence': 81,
              'created_time': '04:39:20',
              'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 16,
 'date': '13/04/2017',
 'hotspots': {'confidence': 81,
              'created

e. Find the top 10 records with the highest surface temperature (°C). 

In [33]:
results = climate.aggregate([
{ "$unwind" : "$hotspots" },
{"$sort":{"hotspots.surface_temperature_celcius":-1}},
{"$limit": 10},
{"$project": {"_id":0}}
])

for doc in results:
    pprint(doc)

{'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:52:00',
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124},
 'max_windspeed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'air_temperature_celcius': 16,
 'date': '4/04/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:32:50',
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123},
 'max_windspeed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': '948701',
 'windspeed_knots': 5.4}
{'air_temperature_celcius': 14,
 'date': '1/05/2017',
 'hotspots': {'confidence': 100,
              'created_time': '04:14:20',
              'latitude': -36.9318,
              'longitude': 143.0907,
              'surface_temperature_celci

f. Find the number of fire in each day.

In [34]:
results = climate.aggregate([
{"$unwind" : "$hotspots"},
{"$group" : { "_id" : '$date', "Total Number of Fires" : {"$sum" : 1}}}
])

for doc in results:
    pprint(doc)

{'Total Number of Fires': 4, '_id': '27/12/2017'}
{'Total Number of Fires': 1, '_id': '25/12/2017'}
{'Total Number of Fires': 4, '_id': '15/12/2017'}
{'Total Number of Fires': 1, '_id': '14/12/2017'}
{'Total Number of Fires': 31, '_id': '30/11/2017'}
{'Total Number of Fires': 3, '_id': '14/11/2017'}
{'Total Number of Fires': 5, '_id': '12/11/2017'}
{'Total Number of Fires': 4, '_id': '11/11/2017'}
{'Total Number of Fires': 2, '_id': '22/11/2017'}
{'Total Number of Fires': 10, '_id': '9/11/2017'}
{'Total Number of Fires': 2, '_id': '8/11/2017'}
{'Total Number of Fires': 4, '_id': '5/11/2017'}
{'Total Number of Fires': 2, '_id': '10/12/2017'}
{'Total Number of Fires': 5, '_id': '27/10/2017'}
{'Total Number of Fires': 1, '_id': '23/10/2017'}
{'Total Number of Fires': 1, '_id': '16/10/2017'}
{'Total Number of Fires': 3, '_id': '15/10/2017'}
{'Total Number of Fires': 3, '_id': '10/10/2017'}
{'Total Number of Fires': 1, '_id': '9/10/2017'}
{'Total Number of Fires': 15, '_id': '16/12/2017'}
{

g. Find the average surface temperature(°C) for each day. You are required to only display average surface temperature(°C) and the date in the output. 

In [35]:
results = climate.aggregate(
   [{"$unwind" : "$hotspots"},
    {"$group":{"_id": "$date",
           "Average Temperature": { "$avg": "$hotspots.surface_temperature_celcius" }
         }
     }
   ]
)

for doc in results:
    pprint(doc)

{'Average Temperature': 62.75, '_id': '27/12/2017'}
{'Average Temperature': 54.0, '_id': '25/12/2017'}
{'Average Temperature': 39.0, '_id': '15/12/2017'}
{'Average Temperature': 70.0, '_id': '14/12/2017'}
{'Average Temperature': 52.41935483870968, '_id': '30/11/2017'}
{'Average Temperature': 52.0, '_id': '14/11/2017'}
{'Average Temperature': 53.0, '_id': '12/11/2017'}
{'Average Temperature': 46.25, '_id': '11/11/2017'}
{'Average Temperature': 61.5, '_id': '22/11/2017'}
{'Average Temperature': 61.3, '_id': '9/11/2017'}
{'Average Temperature': 45.5, '_id': '8/11/2017'}
{'Average Temperature': 58.5, '_id': '5/11/2017'}
{'Average Temperature': 46.0, '_id': '10/12/2017'}
{'Average Temperature': 50.4, '_id': '27/10/2017'}
{'Average Temperature': 38.0, '_id': '23/10/2017'}
{'Average Temperature': 36.0, '_id': '16/10/2017'}
{'Average Temperature': 72.66666666666667, '_id': '15/10/2017'}
{'Average Temperature': 53.333333333333336, '_id': '10/10/2017'}
{'Average Temperature': 44.0, '_id': '9/10/